サンプルを実行する準備

In [ ]:
!pip install openai

In [2]:
import os
os.environ['OPENAI_API_KEY'] = "..."

In [11]:
import openai
import json

functionの定義をします。

In [5]:
function_descriptions = [
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {
                            "type": "string",
                            "description": "The temperature unit to use. Infer this from the users location.",
                            "enum": ["celsius", "fahrenheit"]
                        },
                    },
                    "required": ["location", "unit"],
                },
            }
        ]

質問の内容をuser_queryに格納します。

In [6]:
user_query = "What's the weather like in San Francisco?"

この時点で一度問い合わせを行います。GPTに質問が渡されます。
（まだfunctionは定義されていません）

In [7]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",

        # This is the chat message from the user
        messages=[{"role": "user", "content": user_query}],


        functions=function_descriptions,
        function_call="auto",
    )

GPTは質問に対してどのように判断したのでしょうか？
まず、返答の内容を見てみましょう。

In [8]:
ai_response_message = response["choices"][0]["message"]
print(ai_response_message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"San Francisco, CA\",\n  \"unit\": \"celsius\"\n}"
  }
}


get_current_weatherを呼ぶべしと判断しました。
そのさい、argumentsはlocation,unitを渡すことも判断してくれています。すばらしいですね。
argumentsはJSON形式なので、このままでは実際の値を取得できません。ですので次のようにevalを実行してJSON形式のデータの内容を取得します。

In [9]:
user_location = eval(ai_response_message['function_call']['arguments']).get("location")
user_unit = eval(ai_response_message['function_call']['arguments']).get("unit")

get_current_weather()関数を呼びたいわけですが、この時点では関数は定義されていませんので関数を定義します。  
OPEN AIの公式サイトでは、ここで天気予報取得のAPIを呼ぶように記述されていますが、ここでは関数の動きを見るため固定値を返すテスト用の関数を定義します。

In [12]:
def get_current_weather(location, unit):

    """Get the current weather in a given location"""

    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

実際に関数を呼びます。先に取得したuser_locationとuser_unitをパラメータとしてセットして呼び出します。

In [13]:
function_response = get_current_weather(
    location=user_location,
    unit=user_unit,
)

関数が実行されたその結果を表示しましょう。

In [14]:
function_response

'{"location": "San Francisco, CA", "temperature": "72", "unit": "celsius", "forecast": ["sunny", "windy"]}'

このままでは関数の結果はわかりましたが、文脈にはなっていません。GPTとしての回答を作るため、もう一度GPT APIを呼びます。そのさい、どの関数のレスポンスなのかをnameに与えます。また関数呼び出したあとの内容をcontentに与えます。

In [18]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_query},
        ai_response_message,
        {
            "role": "function",
            "name": "get_current_weather",
            "content": function_response,
        },
    ],
)

APIの返答を表示しましょう。  
きちんと文脈として返答されています。

In [19]:
print (second_response['choices'][0]['message']['content'])

The current weather in San Francisco is sunny and windy with a temperature of 72 degrees Celsius.
